## Optimising Under Goals Markets Early
The aim of this analysis is to understand how to optimise the under goals market when trading in-play. Key questions are:
1. When do the odds begin to drop on average?
2. When is the slope the steepest?
3. Is there are relationship between odds before the match and the early movement in-play. 
4. What is the optimal number of ticks to take to minimise risk? Is this possible to calculate/a sensible question?
5. Can the probability of early goals be predicted with any accuracy?

In [3]:
# Get and clean data
import os
os.chdir('C:\\Users\\rober\\PycharmProjects\\crusher\\src')

from utils.db.database_manager import dbm
from orm.orm import ExchangeOddsSeriesItem, Market, Event, Team, Runner, ExchangeOddsSeries
from crusher.runner import RunnerCodeEnum as RCEnum
from crusher.item_freq_type import ItemFreqTypeCodeEnum as IFTCEnum
from crusher.division import DivisionCodeEnum
from utils.helper_functions import preprocessing as pr
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from dateutil.relativedelta import relativedelta

with dbm.get_managed_session() as session:
    # Get data
    runner = Runner.get_by_code(session, RCEnum.UNDER_2_5_GOALS)
    df = ExchangeOddsSeriesItem.get_series_items_df(session,
                                                    division_code=[DivisionCodeEnum.PREMIER_LEAGUE,
                                                                   DivisionCodeEnum.CHAMPIONSHIP,
                                                                   ],
                                                    runner_uid=runner.runner_uid,
                                                    in_play=True,
                                                    item_freq_type_code=IFTCEnum.MINUTE
                                                    )

df = df.sort_values(['series_uid', 'published_datetime'], ascending=True)

df = pr.drop_series_with_price_jumps(df, 0.2)
df = pr.convert_published_datetime_to_game_time(df)
df = pr.drop_series_by_n_data_points(df, 30)


print(df)



     series_uid  game_time   ltp  series_item_uid  runner_uid in_play  \
0         21794        0.0  1.80              NaN         NaN     NaN   
1         21794        1.0  1.80        6988219.0        40.0    True   
2         21794        2.0  1.79        6988340.0        40.0    True   
3         21794        3.0  1.78        6988267.0        40.0    True   
4         21794        4.0  1.77        6988208.0        40.0    True   
...         ...        ...   ...              ...         ...     ...   
9066      52598       90.0  1.03        8866904.0        40.0    True   
9067      52598       91.0  1.02        8866885.0        40.0    True   
9068      52598       92.0  1.01        8866915.0        40.0    True   
9069      52598       93.0  1.02        8866918.0        40.0    True   
9070      52598       94.0  1.01        8866877.0        40.0    True   

                update_datetime          creation_datetime  
0                           NaT                        NaT  
1

#### TODO:
- Preprocessing as helper functions
- Figure out best way to analyse steepness
- Look into plotly and dash for real time calculations
